In [1]:
import bokeh.models as bmo
import numpy as np
from bokeh.io import output_file, save, output_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show

from tqdm.notebook import tqdm
from fundus_data_toolkit.datamodules.segmentation import (
    IDRiDDataModule_s,
    DDRDataModule_s,
    MESSIDORDataModule_s,
    RETLESDataModule_s,
    FGADRDataModule_s,
    TJDRDataModule_s,
)
from fundus_data_toolkit.datamodules import SEG_PATHS
from fundus_data_toolkit.utils.image_processing import fundus_autocrop
from fundus_lesions_toolkit.constants import (
    lesions2colors,
    labels2names,
)

In [2]:
labels2names, lesions2colors

({0: 'Background',
  1: 'Cotton Wool Spots',
  2: 'Exudates',
  3: 'Hemorrhages',
  4: 'Microaneurysms'},
 {'BG': 'black',
  'CTW': '#eca63f',
  'EX': '#8cf18e',
  'HE': '#4498f0',
  'MA': '#141488'})

In [3]:
def cvt2bokeh(img):
    h, w = img.shape[:2]
    container = np.empty((h, w), dtype=np.uint32)
    view = container.view(dtype=np.uint8).reshape((h, w, 4))
    view[:, :, 0] = img[:, :, 0]  # copy red channelmatch_aspect
    view[:, :, 1] = img[:, :, 1]  # copy blue channel
    view[:, :, 2] = img[:, :, 2]  # copy green channel
    view[:, :, 3] = 255
    return container


datamodules = dict(
    IDRID=IDRiDDataModule_s,
    DDR=DDRDataModule_s,
    MESSIDOR=MESSIDORDataModule_s,
    RETLES=RETLESDataModule_s,
    FGADR=FGADRDataModule_s,
    TJDR=TJDRDataModule_s,
)

for k, d in datamodules.items():
    datamodules[k] = d(SEG_PATHS[k], img_size=512, batch_size=1, num_workers=0)
    datamodules[k].setup_all()

In [9]:
datasets = {name: d.test for name, d in datamodules.items()}
for d in datasets.values():
    d.composer.deactivate_op(5)
datasets["TJDR"].composer.deactivate_op(4)


def plot_image_and_lesions(
    data,
    alpha_mask=0.5,
    size=512,
    title=None,
    x_range=None,
    y_range=None,
    legend=True,
):
    tools = [
        bmo.PanTool(),
        bmo.WheelZoomTool(),
        bmo.BoxZoomTool(match_aspect=True),
        bmo.ResetTool(),
        bmo.FullscreenTool(),
        bmo.CrosshairTool(),
        bmo.SaveTool(),
    ]

    if x_range is None:
        x_range = (0.0, 1.0)
    if y_range is None:
        y_range = (0.0, 1.0)

    image = data["image"][::-1]
    h, w, c = image.shape
    img = cvt2bokeh(image)
    p = figure(
        x_range=x_range,
        y_range=y_range,
        width=size,
        height=size,
        tools=tools,
        title=title,
        match_aspect=True,
    )

    p.image_rgba(image=[img], x=0, y=0, dw=1.0, dh=1.0)

    x, y = np.meshgrid(np.linspace(0, 1.0, h), np.linspace(0, 1.0, w))

    mask = data["mask"][::-1]
    indices = labels2names.keys()
    labels = labels2names.values()
    colors = lesions2colors.values()

    for i, l, c in zip(indices, labels, colors):
        if i == 0:
            continue
        lesion_mask = mask == i
        p.contour(
            x,
            y,
            lesion_mask,
            levels=[0, 1],
            line_width=1,
            line_color=c,
            fill_color=c,
            fill_alpha=alpha_mask,
        )
        if legend:
            p.scatter(
                x=[0], y=[0], legend_label=l, line_color=c, line_width=1, fill_color=c
            )

    # p.legend.orientation = "horizontal"
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks

    p.yaxis.major_tick_line_color = None  # turn off y-axis major ticks
    p.yaxis.minor_tick_line_color = None  # turn off y-axis minor ticks
    p.axis.visible = False

    if legend:
        p.legend.background_fill_alpha = 0.5
    return p

In [5]:
from fundus_lesions_toolkit.models import segment
from fundus_lesions_toolkit.constants import Dataset

models = [
    Dataset.IDRID,
    Dataset.RETLES,
    Dataset.ALL,
]
indices = [17, 200, 1, 11, 8]
all_preds = []
for m in tqdm(models):
    mpred = []
    for index, (dname, dataset) in zip(indices, datasets.items()):
        image = dataset[index]["image"]
        prediction = segment(
            image, train_datasets=m, autofit_resolution=True, reverse_autofit=True
        )
        mask = prediction.argmax(dim=0).cpu().numpy()
        mpred.append((image, mask))

    all_preds.append(mpred)

  0%|          | 0/3 [00:00<?, ?it/s]

/home/clement/Documents/Projets/fundus-lesions-toolkit/src/fundus_lesions_toolkit/models/segmentation.py:195: UserWarning: Setting dropout to 0.2
  warnings.warn(f"Setting dropout to {initial_value}")


In [18]:
output_file("output/results/segment.html")
ps = []
for mname, m in zip(models, all_preds):
    for dname, pred in zip(datasets.keys(), m):
        data = {}
        data["image"] = pred[0]
        data["mask"] = pred[1]
        if not ps:
            ps.append(
                plot_image_and_lesions(
                    data,
                    size=250,
                    alpha_mask=0.15,
                    legend=False,
                    title=f"Trained with {mname.value} \nTested on {dname}",
                )
            )
        else:
            ps.append(
                plot_image_and_lesions(
                    data,
                    size=250,
                    alpha_mask=0.15,
                    x_range=ps[0].x_range,
                    y_range=ps[0].y_range,
                    legend=False,
                    title=f"Trained with {mname.value} \nTested on {dname}",
                )
            )

ps = gridplot(ps, ncols=5, toolbar_options=dict(logo=None))
save(ps)

'/home/clement/Documents/Report/July3/reveal.js/plots/output/results/segment.html'